In [1]:
import sys
import numpy as np
import json
import random
import os
import csv
import time
import re
import cogact_arch as cst
from cogact_arch import CogAct, Node, Link, Modality, Stm

# CSV Categorisation

In [17]:
def get2top_preds(scores_dict): #take a lebels with scores dict and output a list with 2 top pairs, other pairs' scores=0
    scores_list = [item for item in scores_dict.items()]
#     print(scores_list)
    scores_list2 = sorted(scores_list, key=lambda x: -x[1]) #sort scores list by the second item in pair (score), start form largest
#     print(scores_list2)

    total_top2 = scores_list2[0][1] + scores_list2[1][1]

    top2_dict = scores_dict
    for i in range(len(scores_list2)):
        if i < 2:
            top2_dict[scores_list2[i][0]] = np.around( scores_list2[i][1] / total_top2, decimals = 1) #normalise by 2 top scores
        else:
            top2_dict[scores_list2[i][0]] = 0
    top2_list = [item for item in top2_dict.items()]
    top2_list2 = sorted(top2_list, key=lambda x: -x[1])
    
    return top2_list2

In [18]:
""" TOP 2 SCORES ONLY, MAX IMAGE categorisation=============================================================="""
def cat_file(file, get_patterns, correct_label): #
    with open(file, 'r', errors='ignore') as f:
        tests = get_patterns(f.read())

    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')
    print('--', file, '\n')
    print('-------- MAX IMAGE METHOD -------------------------------------\n')
    t_start = time.time()
    labels = {}
    for test in tests:
        max_image = []
        for offset in range(len(test) - 1):
            node = cogact.visual.recognise(test[offset:])
            if node.label is not None:
                if len(node.image) > len(max_image):
                    max_image = node.image
                    # label = cogact.nodes[node.label].image
                    labels[node.label] = labels.get(node.label, 0) + 1
            else:
                for slink in node.similarity_links:
                    if slink.label is not None and len(slink.image) > len(max_image):
                        max_image = slink.image
                        label = slink.label
#                         print('slink label', label)
                        labels[label] = labels.get(label, 0) + 1
        # print('recognises', test, 'as', max_image or None, 'label:', label, sep='\n', end='\n\n')
    

#     lab_sum = sum(labels.values())
#     cogact_normalised = [(k, v / lab_sum) for k, v in labels.items()]
#     top = sorted(cogact_normalised, key=lambda x: -x[1])
    
    print('labels',labels)
    top2preds = get2top_preds(labels) #send labels with scores dict to get top 2 normalised scores
    print('top2preds', top2preds)

    for x in top2preds:
        print('%s (prob = %s)' % (''.join(cogact.nodes[x[0]].image), round(x[1], 1)))

    correct = False
    if top2preds[0][1] == top2preds[1][1]:
        winners = [''.join(cogact.nodes[top2preds[0][0]].image), ''.join(cogact.nodes[top2preds[1][0]].image)]
        correct = correct_label in winners
        print('correct:', correct)
    else:
        winner = ''.join(cogact.nodes[top2preds[0][0]].image) if len(top2preds) > 0 else None
        correct = winner == correct_label
        print('correct:', correct)
    
    print('final labels scores', labels)
    print('ended in', time.time() - t_start, 'sec')

    filename = os.path.basename(file)
    
    return {
        'label': correct_label,
        'file': filename[:filename.rfind('.')],
        **{''.join(cogact.nodes[lbl].image): prob for lbl, prob in top2preds},
        'correct': correct
    }

In [13]:
a = [1,2,5]
b = [1,2,5]
c = a ==b

In [14]:
c

True

In [13]:
"""MAX IMAGE categorisation=============================================================="""
def cat_file(file, get_patterns, correct_label): #
    with open(file, 'r', errors='ignore') as f:
        tests = get_patterns(f.read())

    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')
    print('--', file, '\n')
    print('-------- MAX IMAGE METHOD -------------------------------------\n')
    t_start = time.time()
    labels = {}
    for test in tests:
        max_image = []
        for offset in range(len(test) - 1):
            node = cogact.visual.recognise(test[offset:])
            if node.label is not None:
                if len(node.image) > len(max_image):
                    max_image = node.image
                    # label = cogact.nodes[node.label].image
                    labels[node.label] = labels.get(node.label, 0) + 1
            else:
                for slink in node.similarity_links:
                    if slink.label is not None and len(slink.image) > len(max_image):
                        max_image = slink.image
                        label = slink.label
#                         print('slink label', label)
                        labels[label] = labels.get(label, 0) + 1
        # print('recognises', test, 'as', max_image or None, 'label:', label, sep='\n', end='\n\n')
    

    lab_sum = sum(labels.values())
    cogact_normalised = [(k, v / lab_sum) for k, v in labels.items()]
    top = sorted(cogact_normalised, key=lambda x: -x[1])
    print('labels',labels)
    print('cogact_normalised', cogact_normalised)
    print('top',top)
    for x in top:
        print('%s (prob = %s)' % (''.join(cogact.nodes[x[0]].image), round(x[1], 2)))
    
#     for i in range(len(top)): #CSV write results
#         print(top[i][1])
#         writer.writerow(top[i][1])
        
    winner = ''.join(cogact.nodes[top[0][0]].image) if len(top) > 0 else None
    print('correct:', winner == correct_label)
    
    print('final labels scores', labels)
    print('ended in', time.time() - t_start, 'sec')

    filename = os.path.basename(file)
    return {
        'label': correct_label,
        'file': filename[:filename.rfind('.')],
        **{''.join(cogact.nodes[lbl].image): prob for lbl, prob in cogact_normalised},
        'correct': winner == correct_label
    }

### Dict writer

In [6]:
def write_csv_from_dicts(csv_file, lines):
    with open(csv_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, delimiter=';', fieldnames=lines[0])
        writer.writeheader()
        for line in lines:
            writer.writerow(line)

In [7]:
def cat(folder, get_patterns):
    correct_count = 0
    total_count = 0
    result = []
    for label in os.listdir(folder):
        p = os.path.join(folder, label)
        if os.path.isdir(p):
            for file in os.listdir(p):
                total_count += 1
                res = cat_file(  #use cat_file function to categorise
                    file=os.path.join(p, file),
                    get_patterns=get_patterns,
                    correct_label=label
                )
                result.append(res)
                correct_count += res['correct']
    print('-' * 70)
    print('TOTAL CORRECT:', correct_count, '/', total_count)
    return result

# CATEGORISATION TESTS=====================================

In [19]:
def cat_music(folder): #categorise Max Image WITHOUT using similarity links
    return cat(folder, json.loads)

result = cat_music('json_cuts_predict2')
#write_csv_from_dicts('testing2024/bigRand3.csv', result)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

-- json_cuts_predict2\bach\Bwv0525 Sonate en trio n1_39-100sec.json 

-------- MAX IMAGE METHOD -------------------------------------

labels {4: 30, 13: 14, 7: 12, 28: 16}
top2preds [(4, 0.7), (28, 0.3), (13, 0), (7, 0)]
beethoven (prob = 0.7)
bach (prob = 0.3)
haydn (prob = 0)
mozart (prob = 0)
correct: False
final labels scores {4: 0.7, 13: 0, 7: 0, 28: 0.3}
ended in 0.5276157855987549 sec
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

-- json_cuts_predict2\bach\Bwv0526 Sonate en trio n2_39-100sec.json 

-------- MAX IMAGE METHOD -------------------------------------

labels {7: 10, 28: 27, 4: 27, 13: 8}
top2preds [(28, 0.5), (4, 0.5), (7, 0), (13, 0)]
bach (prob = 0.5)
beethoven (prob = 0.5)
mozart (prob = 0)
haydn (prob = 0)
correct: True
final labels scores {7: 0, 28: 0.5, 4: 0.5, 13: 0}
ended in 1.6206443309783936 sec
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

-- json_cuts_pr

labels {4: 40, 28: 9, 13: 3, 7: 3}
top2preds [(4, 0.8), (28, 0.2), (13, 0), (7, 0)]
beethoven (prob = 0.8)
bach (prob = 0.2)
haydn (prob = 0)
mozart (prob = 0)
correct: False
final labels scores {4: 0.8, 28: 0.2, 13: 0, 7: 0}
ended in 0.6532542705535889 sec
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

-- json_cuts_predict2\mozart\K573 9 Variations for Piano Uber Duport_38-98sec.json 

-------- MAX IMAGE METHOD -------------------------------------

labels {7: 43, 13: 22, 28: 2}
top2preds [(7, 0.7), (13, 0.3), (28, 0)]
mozart (prob = 0.7)
haydn (prob = 0.3)
bach (prob = 0)
correct: True
final labels scores {7: 0.7, 13: 0.3, 28: 0}
ended in 0.6482665538787842 sec
----------------------------------------------------------------------
TOTAL CORRECT: 12 / 20


In [20]:
"""Write CSV!!!!!!!!!!!!!!!!!!!!!!!!!!!!"""
write_csv_from_dicts('participant_DE_MI.csv', result)

# Load LTM

In [8]:
#LOAD TREE STATE
with open('bl_BC.pickle', 'rb') as f:  #rb - read binary
    cogact = pickle.load(f)

In [9]:
cogact.verbal.print_tree()

< ROOT NODE > Node: 1 <  >
-------< m > Node: 4 < m o z a r t > (43986)
-------< b > Node: 6 < b a c h > (14207)
--------------< e > Node: 22 < b e e t h o v e n > (85679)
-------< e > Node: 9 <  >
-------< a > Node: 11 <  >
-------< o > Node: 15 <  >
-------< s > Node: 18 < s c h u b e r t > (8813)
-------< z > Node: 20 <  >
-------< c > Node: 26 <  >
-------< t > Node: 29 <  >
-------< h > Node: 34 <  >
-------< u > Node: 39 <  >
-------< v > Node: 43 <  >
-------< r > Node: 45 <  >
-------< n > Node: 51 <  >
